# 0.0. IMPORTS

In [1]:
import pandas as pd
import inflection
import math

## 0.1. Helper Functions

## 0.2. Loading data

In [2]:
#low_memory = False fala pra função read csv ler todo o arquivo de uma vez só para a memoria
df_sales_raw = pd.read_csv('data/train.csv',low_memory= False)
df_store_raw = pd.read_csv('data/store.csv',low_memory= False)

#merge (primeiro argumento o arquivo referencia + arquivo que será anexado a referencia, 
#depois informo como será feito o merge, no caso left e a coluna que será feito, 
#que deve ser igual em ambos datasets, no caso store)
df_raw = pd.merge (df_sales_raw, df_store_raw, how = 'left', on = 'Store')

In [3]:
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
489925,111,5,2014-04-18,0,0,0,1,b,1,d,c,7890.0,NaN,NaN,1,37.0,2009.0,"Jan,Apr,Jul,Oct"


# 1.0. DESCRICAO DOS DADOS

In [4]:
df1 = df_raw.copy()

## 1.1. Rename Columns

In [5]:
df1.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [6]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
           'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
           'CompetitionDistance', 'CompetitionOpenSinceMonth',
           'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
           'Promo2SinceYear', 'PromoInterval'] 

snakecase = lambda x: inflection.underscore(x)
cols_new = list (map(snakecase,cols_old))

#rename
df1.columns = cols_new


## 1.2. Data Dimensions

In [7]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Cols: {}'.format(df1.shape[1]))

Number of Rows: 1017209
Number of Cols: 18


## 1.3. Data Types

In [8]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [9]:
df1['date'] = pd.to_datetime(df1['date'])

## 1.4. Check NA

In [10]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5. Fillout NA

In [11]:
df1['competition_distance'].max()

75860.0

In [21]:
# competition_distance (quando não tiver uma loja competidora nas redondezas um valor muito grande será adicionado)
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) 
                                                                                   else x)


# competition_open_since_month  (mês e ano que o competidor mais proximo foi aberto)- vou assumir que se não tiver data
    #irei assumir o mesmo mes da variavel date
    #adicionado axis=1 pois ao contrario do comando anterior , irei usar mais de uma coluna, ou seja explicito que será  feito ao longo das colunas
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'],axis=1)     
# competition_open_since_year 
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'],axis=1)
                              
# promo2_since_week  descreve o ano e semana do calendario quando começou a participar da promo 2. Vou pegar o mês de date
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'],axis=1)

# promo2_since_year  
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'],axis=1)


# promo_interval (meses em que a promo 2 ficou ativa)
month_map = {1: 'Jan',2: 'Feb',3: 'Mar',4: 'Apr',5: 'May',6: 'Jun',7: 'Jul',8: 'Aug',9: 'Sep',10: 'Oct',11: 'Nov',12: 'Dec'}

df1['promo_interval'].fillna(0, inplace = True) #inplace = True não retorna valor, faz a modificação direto na coluna
df1['month_map'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval','month_map']].apply(lambda x:0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis = 1)


In [20]:
df1.sample(5).T

,600859,537491,400028,695204,470103
store,660,847,27,230,359
day_of_week,4,5,3,3,2
date,2014-01-09 00:00:00,2014-03-07 00:00:00,2014-07-09 00:00:00,2013-10-16 00:00:00,2014-05-06 00:00:00
sales,5759,7890,6531,5789,7347
customers,617,1079,822,794,605
open,1,1,1,1,1
promo,1,1,0,0,1
state_holiday,0,0,0,0,0
school_holiday,0,0,1,1,0
store_type,a,c,a,d,d


In [22]:
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64